In [6]:
from ImageProcess import *
import cv2
import numpy as np
import os
import TFRtools
import shutil
import tensorflow as tf


In [7]:
#把字符与数字对应，用于ohehot的生成
def key_and_number():
    number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
                'v', 'w', 'x', 'y', 'z']
    ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
                 'V', 'W', 'X', 'Y', 'Z']
    dic = {}
    x = 0
    for i in number:
        dic[i]=x
        x+=1
    for i in alphabet:

        dic[i] = x
        x+=1
    for i in ALPHABET:

        dic[i] = x
        x+=1
    return dic
dic_a = key_and_number()


In [8]:
def Segment4_NUMBER(bin):

    # 去除斑点
    img_bin = RemoveSmallCC(bin, 200, connectivity=4)
    # 形态学腐蚀
    size = 2
    kernel = np.ones((size, size), dtype=np.uint8)
    img_erosion = cv2.erode(img_bin, kernel, iterations=1)
    # 小连通域滤除
    img_erosion = RemoveSmallCC(img_erosion, 20)
    # 形态学膨胀
    img_dila = cv2.dilate(img_erosion, kernel, iterations=1)

    # 滤除横线
    PROJ = BIN_PROJECT(img_dila)  # 投影
    flaw_area = np.where(PROJ < 5)  # 瑕疵区域
    img_dila[:, flaw_area] = 0  # 去除横线
    IMG = RemoveSmallCC(img_dila, 200)  # 去除小连通区域

    # PROJ提取数字部分
    PROJ = BIN_PROJECT(IMG)
    LOC, COUNT = Extract_Num(PROJ)

    # 优化COUNT
    COUNT = COUNT_OPTIMIZE(IMG, LOC, COUNT)

    # (4)分割数字部分
    LOC = Segment4_Num(COUNT, LOC)
    # 分割数字
    NUM0 = IMG[:, LOC[0][0]:LOC[0][1]]
    NUM0 = cv2.resize(NUM0,(32,32))
    NUM0 = RemoveSmallCC(NUM0, 70)
    NUM1 = IMG[:, LOC[1][0]:LOC[1][1]]
    NUM1 = cv2.resize(NUM1, (32, 32))
    NUM1 = RemoveSmallCC(NUM1, 70)
    NUM2 = IMG[:, LOC[2][0]:LOC[2][1]]
    NUM2 = cv2.resize(NUM2, (32, 32))
    NUM2 = RemoveSmallCC(NUM2, 70)
    NUM3 = IMG[:, LOC[3][0]:LOC[3][1]]
    NUM3 = cv2.resize(NUM3, (32, 32))
    NUM3 = RemoveSmallCC(NUM3, 70)
#     print(NUM0)
    return [NUM0,NUM1,NUM2,NUM3]



In [9]:
def GEN_DIR():
    import os
    if not os.path.isdir('NUMBERS'):
        print('文件夹NUMBERS未创建，现在在当前目录下创建..')
        os.mkdir('NUMBERS')
        for i in range(62):
            os.makedirs('./NUMBERS/%d'%i)
    if not os.path.isdir('TEST'):
        print('文件夹TEST未创建，现在在当前目录下创建..')
        os.mkdir('TEST')

In [10]:
def Int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def Bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def Float_frature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def SaveByTFRecord(data, label, prefix, num_shards=5):
    # 变化为数组格式
    data = np.array(data)
    label = np.array(label)
#     print(data[0])
#     print(label)

    # 建立序号
    NUM = np.array(range(0, data.shape[0]))
    print("NUM",NUM)

    # 创建文件夹
    dir, file = os.path.split(prefix)
    if not os.path.isdir(dir):
        print('文件夹%s未创建，现在在当前目录下创建..' % (dir))
        os.mkdir(dir)

    # 分割数据集(0~len-1 ,num_shards等分)
    seg_list = np.int32(np.linspace(0, data.shape[0] - 1, num_shards + 1))
    print("seg_list",seg_list)#分快

    for i in range(num_shards):

        # 生成文件名
        filename = prefix + '-%.4d-%.4d' % (i, num_shards - 1)

        # 写入TFR
        writer = tf.python_io.TFRecordWriter(filename)
        for j in range(seg_list[i], seg_list[i + 1] + 1):
            # 将图像和标签转化成字符串
            image_raw = data[j].tostring()
            label_raw = label[j].tostring()
            # 将图像和标签数据作为一个example
#             print("label_raw:",label_raw)
            example = tf.train.Example(features=tf.train.Features(feature={
                'num': Int64_feature(NUM[j]),
                'image': Bytes_feature(image_raw),
                'label': Bytes_feature(label_raw)
            }))
            writer.write(example.SerializeToString())
        print('已经写入至第%d个文件中第%d项，序号项类型：%s，数据项类型：%s ， 标签项类型：%s' % (i, NUM[j], NUM[j].dtype, data[j].dtype, label[j].dtype))
        writer.close()

    # 游览文件
    fileSets = os.listdir(dir)
    print('成功存储为TFRecord格式！！，%s文件夹下生成文件如下：\n' % (dir))
    print(fileSets)

In [15]:
if __name__ == '__main__':
    img_path = r'./images'# 验证码路径
    img_list = os.listdir(img_path) # 图片列表
    NUM_PATH = r'./NUMBERS' # 训练样本可视化

    # 创建相关目录
    GEN_DIR()

    # 清理NUMBBERS下全部图片
    for i in range(62):
        num_dir = os.path.join(NUM_PATH,str(i))
        num_list = os.listdir(num_dir)
        for num in num_list:
            os.remove(os.path.join(num_dir,num))
        print('成功清理文件夹./NUMBERS/%d下图片'%i)

    for item in os.listdir(r'./TEST'):
        os.remove(os.path.join(r'./TEST',item))
        print('成功清理文件夹./TEST/下图片%s' % item)

    #--------------------------- 随机选取70%作为训练集，%30作为测试集 ---------------------------------------------

    data_total = len(img_list) # 数据集总数
    train_total = int(0.7*data_total) #训练集总量
    test_total = data_total - train_total # 测试集总量

    #随机采样训练样本
    import random
    train_list = random.sample(img_list,train_total) #训练样本列表
    test_list = [item for item in img_list if item not in train_list] # 测试样本列表

    #*************** (1)训练样本分解并写入TFR *****************************
    traindata = []
    trainlabel = []

    step  = 0
    for img_name in train_list:
        prefix = os.path.splitext(img_name)[0]#图片标签名
#         print(prefix)


    
    
        try:
            # 制作数据集
            ##(1)分割数字
            img_gray = cv2.imread(os.path.join(img_path,img_name), flags=cv2.IMREAD_GRAYSCALE)# 读取图片并保存为灰色
            _, img_bin = cv2.threshold(img_gray, int(0.9 * 255), 255, cv2.THRESH_BINARY_INV)  # 二值化阈值系数选取0.9
            numbers = Segment4_NUMBER(img_bin)# 分割数字
    #         print (numbers)
        except BaseException:
            continue
        # 制作标签集
        for n in prefix:
            onehot = np.zeros(shape=62,dtype=np.uint8)
            onehot[dic_a[n]] = 1
            trainlabel.append(onehot) # 加入标签集
        ##(2)保存单个
        for i in range(4):
            
            number = numbers[i]
           # 保存为图片
            savename = prefix+'_%d.jpg'%i
            
            savepath = r'./NUMBERS/%s'%dic_a[prefix[i]]
            cv2.imwrite(os.path.join(savepath,savename),number)
            # 保存traindata
            number = numbers[i]/255 # 归一化
            number = np.reshape(number, [32 * 32])  # 平铺
            traindata.append(number)
        step+=1
#     print (len(trainlabel))
#     print (trainlabel[0])

        print('成功分割第%d/%d张训练验证码:%s'%(step,len(train_list),prefix))

    ## 训练样本存入TFR
    SaveByTFRecord(traindata,trainlabel,r'./TFR/CODE_TRAIN',5)
    
    #*************************** (2)测试样本分解写入TFR ******************************************************

    testdata = []
    testlabel = []

    step = 0
    for img_name in test_list:
        #图片标签名
        prefix = os.path.splitext(img_name)[0]

        # 制作标签集
        for n in prefix:
            onehot = np.zeros(shape=10, dtype=np.uint8)
            onehot[int(n)] = 1
            testlabel.append(onehot)  # 加入标签集

        # 制作数据集
        ## （1）分割数字
        img_gray = cv2.imread(os.path.join(img_path, img_name), flags=cv2.IMREAD_GRAYSCALE)  # 读取图片并保存为灰色
        _, img_bin = cv2.threshold(img_gray, int(0.9 * 255), 255, cv2.THRESH_BINARY_INV)  # 二值化阈值系数选取0.9
        numbers = Segment4_NUMBER(img_bin)  # 分割数字

        ##（2）保存数字
        for i in range(4):
            # 保存testdata
            number = numbers[i] / 255  # 归一化
            number = np.reshape(number, [32 * 32])  # 平铺
            testdata.append(number)

        ## (3)复制图片
        src = os.path.join(img_path,img_name)
        dst = os.path.join(r'./TEST',img_name)
        shutil.copy(src,dst)

        step += 1
        print('成功分割第%d/%d张测试验证码:%s' % (step, len(test_list), prefix))

    ## 测试样本存入TFR
    TFRtools.SaveByTFRecord(testdata,testlabel,r'./TFR/CODE_TEST',5)



成功清理文件夹./NUMBERS/0下图片
成功清理文件夹./NUMBERS/1下图片
成功清理文件夹./NUMBERS/2下图片
成功清理文件夹./NUMBERS/3下图片
成功清理文件夹./NUMBERS/4下图片
成功清理文件夹./NUMBERS/5下图片
成功清理文件夹./NUMBERS/6下图片
成功清理文件夹./NUMBERS/7下图片
成功清理文件夹./NUMBERS/8下图片
成功清理文件夹./NUMBERS/9下图片
成功清理文件夹./NUMBERS/10下图片
成功清理文件夹./NUMBERS/11下图片
成功清理文件夹./NUMBERS/12下图片
成功清理文件夹./NUMBERS/13下图片
成功清理文件夹./NUMBERS/14下图片
成功清理文件夹./NUMBERS/15下图片
成功清理文件夹./NUMBERS/16下图片
成功清理文件夹./NUMBERS/17下图片
成功清理文件夹./NUMBERS/18下图片
成功清理文件夹./NUMBERS/19下图片
成功清理文件夹./NUMBERS/20下图片
成功清理文件夹./NUMBERS/21下图片
成功清理文件夹./NUMBERS/22下图片
成功清理文件夹./NUMBERS/23下图片
成功清理文件夹./NUMBERS/24下图片
成功清理文件夹./NUMBERS/25下图片
成功清理文件夹./NUMBERS/26下图片
成功清理文件夹./NUMBERS/27下图片
成功清理文件夹./NUMBERS/28下图片
成功清理文件夹./NUMBERS/29下图片
成功清理文件夹./NUMBERS/30下图片
成功清理文件夹./NUMBERS/31下图片
成功清理文件夹./NUMBERS/32下图片
成功清理文件夹./NUMBERS/33下图片
成功清理文件夹./NUMBERS/34下图片
成功清理文件夹./NUMBERS/35下图片
成功清理文件夹./NUMBERS/36下图片
成功清理文件夹./NUMBERS/37下图片
成功清理文件夹./NUMBERS/38下图片
成功清理文件夹./NUMBERS/39下图片
成功清理文件夹./NUMBERS/40下图片
成功清理文件夹./NUMBERS/41下图片
成功清理文件夹./NUMBERS/42下图片
成功清理文件夹./NUMBERS/43下图

成功分割第266/13889张训练验证码:RcGB
成功分割第267/13889张训练验证码:pVSj
成功分割第268/13889张训练验证码:Vvvv
成功分割第269/13889张训练验证码:3Duq
成功分割第270/13889张训练验证码:EO9O
成功分割第271/13889张训练验证码:kiei
成功分割第272/13889张训练验证码:O7GB
成功分割第273/13889张训练验证码:XgfI
成功分割第274/13889张训练验证码:HpNR
成功分割第275/13889张训练验证码:YZoQ
成功分割第276/13889张训练验证码:4hid
成功分割第277/13889张训练验证码:HzbC
成功分割第278/13889张训练验证码:D3Um
成功分割第279/13889张训练验证码:ZmkZ
成功分割第280/13889张训练验证码:pl5G
成功分割第281/13889张训练验证码:0osN
成功分割第282/13889张训练验证码:uobY
成功分割第283/13889张训练验证码:YUc9
成功分割第284/13889张训练验证码:CIAL
成功分割第285/13889张训练验证码:dsE5
成功分割第286/13889张训练验证码:759j
成功分割第287/13889张训练验证码:loHI
成功分割第288/13889张训练验证码:u0uf
成功分割第289/13889张训练验证码:0uVo
成功分割第290/13889张训练验证码:gzqF
成功分割第291/13889张训练验证码:KOwk
成功分割第292/13889张训练验证码:2Puo
成功分割第293/13889张训练验证码:pd6s
成功分割第294/13889张训练验证码:cfcz
成功分割第295/13889张训练验证码:Uoqh
成功分割第296/13889张训练验证码:HUM3
成功分割第297/13889张训练验证码:ZMBZ
成功分割第298/13889张训练验证码:hU6K
成功分割第299/13889张训练验证码:J5SE
成功分割第300/13889张训练验证码:OxgN
成功分割第301/13889张训练验证码:ra2e
成功分割第302/13889张训练验证码:bERI
成功分割第303/13889张训练验证码:be6K
成功分割第304/138

成功分割第583/13889张训练验证码:kqvZ
成功分割第584/13889张训练验证码:9aAg
成功分割第585/13889张训练验证码:goDr
成功分割第586/13889张训练验证码:HwfR
成功分割第587/13889张训练验证码:zebF
成功分割第588/13889张训练验证码:Vd9t
成功分割第589/13889张训练验证码:iJBm
成功分割第590/13889张训练验证码:o4Gn
成功分割第591/13889张训练验证码:ZQWA
成功分割第592/13889张训练验证码:ob5G
成功分割第593/13889张训练验证码:TD1J
成功分割第594/13889张训练验证码:2VCh
成功分割第595/13889张训练验证码:rafz
成功分割第596/13889张训练验证码:dJQo
成功分割第597/13889张训练验证码:5SMt
成功分割第598/13889张训练验证码:Ac7S
成功分割第599/13889张训练验证码:Pc8F
成功分割第600/13889张训练验证码:bWTx
成功分割第601/13889张训练验证码:ULV1
成功分割第602/13889张训练验证码:8kL2
成功分割第603/13889张训练验证码:7IDk
成功分割第604/13889张训练验证码:bfEW
成功分割第605/13889张训练验证码:V1jp
成功分割第606/13889张训练验证码:57jj
成功分割第607/13889张训练验证码:eZmx
成功分割第608/13889张训练验证码:aNFd
成功分割第609/13889张训练验证码:Suds
成功分割第610/13889张训练验证码:QUUk
成功分割第611/13889张训练验证码:Zmoz
成功分割第612/13889张训练验证码:uyOF
成功分割第613/13889张训练验证码:ayfK
成功分割第614/13889张训练验证码:g90E
成功分割第615/13889张训练验证码:ygX5
成功分割第616/13889张训练验证码:rl8U
成功分割第617/13889张训练验证码:udKx
成功分割第618/13889张训练验证码:tc1o
成功分割第619/13889张训练验证码:j4cY
成功分割第620/13889张训练验证码:PzHC
成功分割第621/138

成功分割第900/13889张训练验证码:iEZt
成功分割第901/13889张训练验证码:mRtq
成功分割第902/13889张训练验证码:qLw0
成功分割第903/13889张训练验证码:Fv0b
成功分割第904/13889张训练验证码:OQ3e
成功分割第905/13889张训练验证码:TWY6
成功分割第906/13889张训练验证码:uQk1
成功分割第907/13889张训练验证码:dqhS
成功分割第908/13889张训练验证码:giMz
成功分割第909/13889张训练验证码:fffO
成功分割第910/13889张训练验证码:7Tu9
成功分割第911/13889张训练验证码:udDX
成功分割第912/13889张训练验证码:WyyJ
成功分割第913/13889张训练验证码:7LmX
成功分割第914/13889张训练验证码:JKSo
成功分割第915/13889张训练验证码:FbS6
成功分割第916/13889张训练验证码:pPbf
成功分割第917/13889张训练验证码:6hS7
成功分割第918/13889张训练验证码:WPPR
成功分割第919/13889张训练验证码:droz
成功分割第920/13889张训练验证码:78Tb
成功分割第921/13889张训练验证码:OxDE
成功分割第922/13889张训练验证码:sT7F
成功分割第923/13889张训练验证码:15oH
成功分割第924/13889张训练验证码:qp0X
成功分割第925/13889张训练验证码:mmmC
成功分割第926/13889张训练验证码:6ZZH
成功分割第927/13889张训练验证码:Y9Ga
成功分割第928/13889张训练验证码:Km9T
成功分割第929/13889张训练验证码:xS6T
成功分割第930/13889张训练验证码:M9tE
成功分割第931/13889张训练验证码:k7Qe
成功分割第932/13889张训练验证码:uNfT
成功分割第933/13889张训练验证码:Wcl6
成功分割第934/13889张训练验证码:iTd8
成功分割第935/13889张训练验证码:x4n5
成功分割第936/13889张训练验证码:KzdH
成功分割第937/13889张训练验证码:gHxq
成功分割第938/138

成功分割第1209/13889张训练验证码:di2f
成功分割第1210/13889张训练验证码:4wwo
成功分割第1211/13889张训练验证码:e6Aj
成功分割第1212/13889张训练验证码:demA
成功分割第1213/13889张训练验证码:SU2F
成功分割第1214/13889张训练验证码:wDS7
成功分割第1215/13889张训练验证码:wg5I
成功分割第1216/13889张训练验证码:Jk9F
成功分割第1217/13889张训练验证码:aU3b
成功分割第1218/13889张训练验证码:gmZH
成功分割第1219/13889张训练验证码:L9tu
成功分割第1220/13889张训练验证码:k3Yx
成功分割第1221/13889张训练验证码:1toI
成功分割第1222/13889张训练验证码:V4O3
成功分割第1223/13889张训练验证码:zzAm
成功分割第1224/13889张训练验证码:8j6E
成功分割第1225/13889张训练验证码:ixgz
成功分割第1226/13889张训练验证码:t3Mz
成功分割第1227/13889张训练验证码:r27J
成功分割第1228/13889张训练验证码:NhWL
成功分割第1229/13889张训练验证码:jJWn
成功分割第1230/13889张训练验证码:ya25
成功分割第1231/13889张训练验证码:zSom
成功分割第1232/13889张训练验证码:K3bY
成功分割第1233/13889张训练验证码:IrWr
成功分割第1234/13889张训练验证码:b4rA
成功分割第1235/13889张训练验证码:vvFd
成功分割第1236/13889张训练验证码:Yo2q
成功分割第1237/13889张训练验证码:e0mA
成功分割第1238/13889张训练验证码:Tll0
成功分割第1239/13889张训练验证码:TVq8
成功分割第1240/13889张训练验证码:cgem
成功分割第1241/13889张训练验证码:EmYi
成功分割第1242/13889张训练验证码:JAnn
成功分割第1243/13889张训练验证码:19HD
成功分割第1244/13889张训练验证码:7jMI
成功分割第1245/13889张训练验证码:9IkB
成

...

成功分割第12690/13889张训练验证码:o5Ub
成功分割第12691/13889张训练验证码:SP9L
成功分割第12692/13889张训练验证码:Uc0K
成功分割第12693/13889张训练验证码:Al9i
成功分割第12694/13889张训练验证码:KoiC
成功分割第12695/13889张训练验证码:AhAK
成功分割第12696/13889张训练验证码:Ocaw
成功分割第12697/13889张训练验证码:Ffrf
成功分割第12698/13889张训练验证码:pSMx
成功分割第12699/13889张训练验证码:wwDl
成功分割第12700/13889张训练验证码:v6Bj
成功分割第12701/13889张训练验证码:sLaY
成功分割第12702/13889张训练验证码:2RAD
成功分割第12703/13889张训练验证码:knZA
成功分割第12704/13889张训练验证码:emt2
成功分割第12705/13889张训练验证码:Q06j
成功分割第12706/13889张训练验证码:g95j
成功分割第12707/13889张训练验证码:YBOr
成功分割第12708/13889张训练验证码:uDmX
成功分割第12709/13889张训练验证码:wX2a
成功分割第12710/13889张训练验证码:Nig3
成功分割第12711/13889张训练验证码:4r4e
成功分割第12712/13889张训练验证码:m1iX
成功分割第12713/13889张训练验证码:xlRv
成功分割第12714/13889张训练验证码:UCmL
成功分割第12715/13889张训练验证码:oo5t
成功分割第12716/13889张训练验证码:q6JL
成功分割第12717/13889张训练验证码:S6A3
成功分割第12718/13889张训练验证码:DwXg
成功分割第12719/13889张训练验证码:bwu1
成功分割第12720/13889张训练验证码:EZGs
成功分割第12721/13889张训练验证码:zTtJ
成功分割第12722/13889张训练验证码:HvJH
成功分割第12723/13889张训练验证码:1a6F
成功分割第12724/13889张训练验证码:A09B
成功分割第12725/13889张训练验

成功分割第12984/13889张训练验证码:8mc8
成功分割第12985/13889张训练验证码:78Au
成功分割第12986/13889张训练验证码:S2JB
成功分割第12987/13889张训练验证码:yeKV
成功分割第12988/13889张训练验证码:55N0
成功分割第12989/13889张训练验证码:fSr2
成功分割第12990/13889张训练验证码:DcyQ
成功分割第12991/13889张训练验证码:g7yH
成功分割第12992/13889张训练验证码:sEPx
成功分割第12993/13889张训练验证码:YvRc
成功分割第12994/13889张训练验证码:6m0l
成功分割第12995/13889张训练验证码:amAM
成功分割第12996/13889张训练验证码:ChDi
成功分割第12997/13889张训练验证码:gYUK
成功分割第12998/13889张训练验证码:FCfF
成功分割第12999/13889张训练验证码:licG
成功分割第13000/13889张训练验证码:cOND
成功分割第13001/13889张训练验证码:dihK
成功分割第13002/13889张训练验证码:FjQ2
成功分割第13003/13889张训练验证码:KjBa
成功分割第13004/13889张训练验证码:HmLY
成功分割第13005/13889张训练验证码:FArT
成功分割第13006/13889张训练验证码:vLPA
成功分割第13007/13889张训练验证码:EPwg
成功分割第13008/13889张训练验证码:juPK
成功分割第13009/13889张训练验证码:TkMC
成功分割第13010/13889张训练验证码:X3uP
成功分割第13011/13889张训练验证码:fg9P
成功分割第13012/13889张训练验证码:j58y
成功分割第13013/13889张训练验证码:AIDh
成功分割第13014/13889张训练验证码:RyLA
成功分割第13015/13889张训练验证码:2mAB
成功分割第13016/13889张训练验证码:ULM4
成功分割第13017/13889张训练验证码:WyHk
成功分割第13018/13889张训练验证码:jeHw
成功分割第13019/13889张训练验

成功分割第13277/13889张训练验证码:EZGb
成功分割第13278/13889张训练验证码:oEmn
成功分割第13279/13889张训练验证码:DjLu
成功分割第13280/13889张训练验证码:nLgU
成功分割第13281/13889张训练验证码:k9KL
成功分割第13282/13889张训练验证码:pgSv
成功分割第13283/13889张训练验证码:BIoS
成功分割第13284/13889张训练验证码:PtBr
成功分割第13285/13889张训练验证码:BxuB
成功分割第13286/13889张训练验证码:oRUa
成功分割第13287/13889张训练验证码:Emrt
成功分割第13288/13889张训练验证码:bynw
成功分割第13289/13889张训练验证码:ExSV
成功分割第13290/13889张训练验证码:7Woo
成功分割第13291/13889张训练验证码:EXdr
成功分割第13292/13889张训练验证码:mX3f
成功分割第13293/13889张训练验证码:M5Ly
成功分割第13294/13889张训练验证码:yyaY
成功分割第13295/13889张训练验证码:nm3x
成功分割第13296/13889张训练验证码:jK06
成功分割第13297/13889张训练验证码:4tzP
成功分割第13298/13889张训练验证码:dAFL
成功分割第13299/13889张训练验证码:xwvY
成功分割第13300/13889张训练验证码:ED3T
成功分割第13301/13889张训练验证码:gGGW
成功分割第13302/13889张训练验证码:EI9w
成功分割第13303/13889张训练验证码:WeOf
成功分割第13304/13889张训练验证码:35Ad
成功分割第13305/13889张训练验证码:m3w0
成功分割第13306/13889张训练验证码:OB13
成功分割第13307/13889张训练验证码:9mpq
成功分割第13308/13889张训练验证码:2vDI
成功分割第13309/13889张训练验证码:KPsS
成功分割第13310/13889张训练验证码:f3fS
成功分割第13311/13889张训练验证码:2iP1
成功分割第13312/13889张训练验

成功分割第13570/13889张训练验证码:4tl0
成功分割第13571/13889张训练验证码:hFuw
成功分割第13572/13889张训练验证码:rkfb
成功分割第13573/13889张训练验证码:PO4Z
成功分割第13574/13889张训练验证码:dpeC
成功分割第13575/13889张训练验证码:4dzk
成功分割第13576/13889张训练验证码:Ijf7
成功分割第13577/13889张训练验证码:pYll
成功分割第13578/13889张训练验证码:n8wd
成功分割第13579/13889张训练验证码:zhFI
成功分割第13580/13889张训练验证码:hNaf
成功分割第13581/13889张训练验证码:EgZf
成功分割第13582/13889张训练验证码:jWeK
成功分割第13583/13889张训练验证码:A1TK
成功分割第13584/13889张训练验证码:Zqtm
成功分割第13585/13889张训练验证码:alzU
成功分割第13586/13889张训练验证码:MyY2
成功分割第13587/13889张训练验证码:rdYy
成功分割第13588/13889张训练验证码:OhGT
成功分割第13589/13889张训练验证码:QT17
成功分割第13590/13889张训练验证码:Hlqv
成功分割第13591/13889张训练验证码:FqpC
成功分割第13592/13889张训练验证码:qZv3
成功分割第13593/13889张训练验证码:T1zz
成功分割第13594/13889张训练验证码:nalA
成功分割第13595/13889张训练验证码:AD6b
成功分割第13596/13889张训练验证码:iBaz
成功分割第13597/13889张训练验证码:aU0F
成功分割第13598/13889张训练验证码:4PgR
成功分割第13599/13889张训练验证码:nRB6
成功分割第13600/13889张训练验证码:WGU9
成功分割第13601/13889张训练验证码:CRn0
成功分割第13602/13889张训练验证码:euv2
成功分割第13603/13889张训练验证码:tOwR
成功分割第13604/13889张训练验证码:EVHR
成功分割第13605/13889张训练验

成功分割第13863/13889张训练验证码:Zzoe
成功分割第13864/13889张训练验证码:oAGi
成功分割第13865/13889张训练验证码:beky
成功分割第13866/13889张训练验证码:LIMH
成功分割第13867/13889张训练验证码:rlGD
成功分割第13868/13889张训练验证码:faMW
成功分割第13869/13889张训练验证码:whEZ
成功分割第13870/13889张训练验证码:N8j9
成功分割第13871/13889张训练验证码:cDfm
成功分割第13872/13889张训练验证码:BWVi
成功分割第13873/13889张训练验证码:ez4P
成功分割第13874/13889张训练验证码:DAnI
成功分割第13875/13889张训练验证码:AWXM
成功分割第13876/13889张训练验证码:9DyK
成功分割第13877/13889张训练验证码:aE6T
成功分割第13878/13889张训练验证码:TiME
成功分割第13879/13889张训练验证码:nwlB
成功分割第13880/13889张训练验证码:Os5U
成功分割第13881/13889张训练验证码:31xU
成功分割第13882/13889张训练验证码:aqnh
成功分割第13883/13889张训练验证码:e4Nl
成功分割第13884/13889张训练验证码:kphq
成功分割第13885/13889张训练验证码:xJnX
成功分割第13886/13889张训练验证码:pRZJ
成功分割第13887/13889张训练验证码:HXTZ
成功分割第13888/13889张训练验证码:uVhk
NUM [    0     1     2 ... 55549 55550 55551]
文件夹./TFR未创建，现在在当前目录下创建..
seg_list [    0 11110 22220 33330 44440 55551]
已经写入至第0个文件中第11110项，序号项类型：int32，数据项类型：float64 ， 标签项类型：uint8
已经写入至第1个文件中第22220项，序号项类型：int32，数据项类型：float64 ， 标签项类型：uint8
已经写入至第2个文件中第33330项，序号项类型：int32，数据项类型